In [57]:
import requests 
from bs4 import BeautifulSoup
from time import sleep

In [99]:
BASE_URL = 'https://health.mail.ru'
mongo_client = get_mongo_client(MONGO_HOST, MONGO_PORT, MONGO_USER, MONGO_PASSWORD)

In [48]:
themes_links = []
themes_request = requests.get('https://health.mail.ru/consultation/')
themes_soup = BeautifulSoup(themes_request.content.decode('utf-8'), 'html.parser')
for theme_link in themes_soup.findAll("a", { "class" : "p-rubric-list__title link-holder" }):
    themes_links.append(theme_link['href'])

In [51]:
len(themes_links), themes_links[0]

(40, '/consultation/list/rubric/allergology/')

In [108]:
type(requests.get('https://health.mail.ru/consultation/list/rubric/stomatology/?page=955').status_code)

int

In [ ]:
for theme in themes_links:
    print(theme)
    theme_name = theme.split('/')[-2]
    
    print(theme_name)
    for i in range(1, 3600):
        page_link = BASE_URL + theme + '?page=' + str(i)
        print(page_link)
        req = requests.get(page_link)
        req_soup = BeautifulSoup(req.content.decode('utf-8'), 'html.parser')
        
        if not req.ok:
            if req.status_code == 404:
                print('pages ended, ogo!')
                break
            print('something not ok')
            sleep(25)
            continue
            
        for i in req_soup.findAll("a", { "class" : "entry__link link-holder" }):
            consult_req = requests.get(BASE_URL + i['href'])
            consult_soup = BeautifulSoup(consult_req.content.decode('utf-8'), 'html.parser')
            data = [j.text for j in consult_soup.findAll("div", { "class" : "entry__description" })]
            question = data[0]
            answer = data[1]
            href = i['href']
            mongo_client[MONGO_DB_NAME][MONGO_COLLECTION_NAME].insert_one(
                {
                    'question': question,
                    'answer': answer,
                    'href': href,
                    'theme_name': theme_name,
                    'page_link': page_link,
                    'theme_link': theme
                }
            )
            del consult_req
            del consult_soup
        print(req.ok)
        sleep(5)
    sleep(2)

/consultation/list/rubric/allergology/
allergology
https://health.mail.ru/consultation/list/rubric/allergology/?page=10
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=11
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=12
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=13
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=14
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=15
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=16
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=17
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=18
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=19
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=20
True
https://health.mail.ru/consultation/list/rubric/allergology/?page=21
True
https://health.mail.ru/consultation/list/rubric/allergology/?